In [1]:
%matplotlib inline

import pickle
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.optimizers import RMSprop
import pandas as pd
import os

dataset = pd.read_csv('dataAccelGyroX2.txt')
dataset.head()

,-5700,6888,12384,420,-5707,-1692,-5980,6384,12776,794,...,1594,-8823,-4679,-2184,14992,1132,1294,-4062,-1328,0
0,-5980,6384,12776,794,-1036,-1222,-7348,7424,13836,730,...,1294,-4062,-1328,-1380,18304,1952,-906,-2616,1565,0
1,-7348,7424,13836,730,-2235,389,-5584,6744,13868,109,...,-906,-2616,1565,-1512,16124,2680,-753,2345,2680,0
2,-5584,6744,13868,109,-2177,-1112,-6224,6480,14060,-137,...,-753,2345,2680,-1576,14240,1920,-1713,550,3010,0
3,-6224,6480,14060,-137,1387,593,-6492,6288,13244,941,...,-1713,550,3010,-932,14812,2484,-1549,989,1968,0
4,-6492,6288,13244,941,-1288,-1612,-6760,6200,13648,421,...,-1549,989,1968,-1524,16360,2480,-677,3366,3260,0


In [2]:
# Spliting the points and the labels
X = dataset.iloc[:, :-1].values  
y = dataset.iloc[:, -1].values

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [4]:
window = 30 # depends on time window
epochs = 15
batch_size = 128
acc_vec_dim = 6 # depends on how many sensors used

class_names = list(set(y))
num_class = len(class_names)

X_testg = X_test.reshape(X_test.shape[0], window, acc_vec_dim)
X_traing = X_train.reshape(X_train.shape[0], window, acc_vec_dim)
print(X_traing.shape)
print(X_testg.shape)
print(X_traing[0])

(625060, 30, 6)
(156266, 30, 6)
[[ -6515  17771  10199   8500  -8967 -12133]
 [ -6639  15779   9472   6834   -338 -15396]
 [ -7537  10579  10477   4952  -5231 -13073]
 [ -7034   9373   6904  -1202  -9545 -10199]
 [ -5612   9075   5968  -8274  -6009   2433]
 [ -6129   6243   6188 -10572  -2764  10781]
 [ -7546  12582   4439 -15846  14293  22125]
 [ -6305   5945   6261  -7177  25612  19700]
 [-20327   5711  10741  22672 -20365   8025]
 [ -7202  16461  13710   8504 -27374 -18463]
 [ -3173  19923   6343  -9565 -18708 -18090]
 [ -4559   1680   6399  -3679  16744  -1461]
 [ -6387   5480  13223  12296    493   4683]
 [  9183   -826   -747  -3702 -22531 -31679]
 [ -1133  -5656  -4816 -22679    203 -16818]
 [-31343  17166  12929 -37601   9521  13529]
 [-22123  17304  27193 -23652  24131   7134]
 [-19584  11510  12006  -7867   2972 -14617]
 [  2084    197  11752 -23617   5667  -8238]
 [-17615  -2394  -3545 -36344  16120  18617]
 [-16509  -3972  -2918 -38600   6189   -838]
 [ 12947  11000 -14094 

In [5]:
model = Sequential()
model.add(LSTM(512,dropout=0.15,recurrent_dropout=0.15,input_shape=(window, acc_vec_dim)))
model.add(Dropout(0.15))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(len(class_names), activation='softmax'))
print(model.summary())

checkpoint_path = "trainingAG_LSTM5/cp.ckpt" #MAKE SURE CHECK SINI DULU............................
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

#model.load_weights(checkpoint_path)

history = model.fit(X_traing, y_train,
                        batch_size=batch_size,
                        epochs=15,
                        verbose=1,
                        validation_data=(X_testg, y_test),
                        callbacks=[cp_callback])

W0815 13:14:14.117362  6664 recurrent_v2.py:1098] Layer lstm will not use cuDNN kernel since it doesn't meet the cuDNN kernel criteria. It will use generic GPU kernel as fallback when running on GPU


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               1062912   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 1

KeyboardInterrupt: 